In [ ]:
from sklearn.neural_network import MLPClassifier

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
x = pd.read_csv('ml-25m/pca_final.csv')
y = pd.read_csv('ml-25m/rating_only.csv')

In [ ]:
def run_MLPClassifier(x, y, hidden_layer_sizes, activation, solver, learning_rate):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, learning_rate_init=learning_rate)
    model.fit(x, y)
    y_pred = model.predict(x_test)
    print(classification_report(y_test, y_pred))

    with open('MLP\\reportMLPClassifier%s%d%s%0.0f.txt' %(activation, hidden_layer_sizes,solver,learning_rate), 'w') as f:
        print(classification_report(y_test, y_pred), file=f)

    plt.figure(figsize=(10,10))
    plt.matshow(confusion_matrix(y_test, y_pred),fignum=1)
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=['bad', 'good', 'ok'], yticklabels=['bad', 'good', 'ok'])
    plt.title('Confusion Matrix for MLPClassifier Size: %d Activation: %s solver: %s L_rate %0.2f' %(hidden_layer_sizes, activation, solver, learning_rate))
    plt.savefig('MLP\confusion_matrixMLPClassifier%s%d%s%0.0f.png' %(activation, hidden_layer_sizes,solver,learning_rate))
    plt.show()

    unique, counts = np.unique(y, return_counts=True)

    y = label_binarize(y, classes=unique)
    n_classes = len(unique)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    clf = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, learning_rate_init=learning_rate))
    y_score = clf.fit(x_train, y_train).predict_proba(x_test)

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])


    fig, ax = plt.subplots(figsize=(10,10))
    ax.plot([0, 1], [0, 1], 'k--')
    ax.axis(xmin=0, xmax=1, ymin=0, ymax=1.05)
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic for MLPClassifier for Size: %d Activation: %s solver: %s L_rate %0.2f' %(hidden_layer_sizes, activation, solver, learning_rate))
    # Plot of a ROC curve for a specific class
    for i in range(n_classes):
        
        ax.plot(fpr[i], tpr[i], label='ROC curve for class %s (area = %0.2f)' % (unique[i], roc_auc[i]))
        
    ax.legend(loc="lower right")
    fig.savefig('MLP\\roc_curveMLPClassifier%s%d%s%0.0f.png' %(activation, hidden_layer_sizes,solver,learning_rate))
    plt.show()

In [ ]:
for hidden_layer_sizes in [100, 500, 1000]:
    for activation in ['logistic', 'tanh', 'relu']:
        for solver in ['lbfgs', 'sgd', 'adam']: 
            run_MLPClassifier(x, y, hidden_layer_sizes, activation, solver, 0.1)